# One-dimensional buyancy space adv-diffusion for diagnostics
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import shutil
import os
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
from mpi4py import MPI
from scipy.special import erf
from scipy.interpolate import spline

import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

from dedalus.tools import post
import pathlib
from dedalus.extras import plot_tools

In [ ]:
# 2D run data:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
run = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400/'];
params = np.load(run[0] + 'runparams.npz');locals().update(params)
b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
ba = (b[:-1]+b[1:])/2

with h5py.File(run[0] + 'ifields.h5', mode='r') as file:
    B = file['tasks']['B'][0,:,:]
    B = (B[1:,:] + B[:-1,:])/2
    B = (B[:,1:] + B[:,:-1])/2
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

with h5py.File(run[0] + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    trF = np.zeros((len(b)-1,len(t)))
    for i in range(0,len(t)):#len(t),pfr):
        tr = file['tasks']['tr'][i,:,:]
        tr = (tr[1:,:] + tr[:-1,:])/2
        tr = (tr[:,1:] + tr[:,:-1])/2
        tr = tr*dA
        trF[:,i] = [np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]]

zF = (b[1:]+b[:-1])/2/N2;
dzF = zF[1]-zF[0]
tF = t;

imax = np.argmax(trF[:,0])
zF = zF - zF[imax]
trF = trF/np.sum(trF[imax,0]*dzF)

In [ ]:
# Simple 3-parameter moments method:
DT = 5*360. # estimate after one-year

basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
run = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400/'];
params = np.load(run[0] + 'runparams.npz');locals().update(params)

with h5py.File(run[0] + 'snapshots.h5', mode='r') as file:
    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    zm1 = file['tasks']['bm1i'][:,0,0]/N2
    zm2 = file['tasks']['bm2i'][:,0,0]/N2/N2
    zm3 = file['tasks']['bm3i'][:,0,0]/N2/N2/N2
    zm4 = file['tasks']['bm4i'][:,0,0]/N2/N2/N2/N2

imax = np.argmin(abs(t-DT));

A = (zm1[imax]-zm1[0])/trT[0]/(DT*lday) # wd (1st 2 moments), or wd - dK/dz (2nd 2 moments)

# Just first two moments (no dK/dz):
wd = A
K0 = (zm2[imax]-zm2[0] - 2*A*DT*lday*zm1[0]-A*A*trT[0]*DT*DT*lday*lday)/(2*DT*lday*trT[0])



# # B = D + E*K0:
# D = (zm2[imax]-zm2[0])/(2*DT*lday*zm1[0]+A*trT[0]*DT*DT*lday*lday)
# E = -(2*DT*lday*trT[0])/(2*DT*lday*zm1[0]+A*trT[0]*DT*DT*lday*lday)

# #

# Eq1 = (zm2[imax]-zm2[0]) = K0*(2*DT*lday*trT[0]) + B*(2*DT*lday*zm1[0]+A*trT[0]*DT*DT*lday*lday)
# Eq2 = (zm3[imax]-zm3[0]) = 


In [ ]:
# Input Grids and parameters ------------------------------------------------------------
Lh = 3200.
h_basis = de.Fourier('h', 128, interval=(-Lh/2,Lh/2))#, dealias=3/2)
domain = de.Domain([h_basis], grid_dtype=np.float64)
h = domain.grid(0)

N2 = 1.0e-6
#wd = 5.e-7#1.0e-7
#K0 = 2e-4
Kh = 0.#0.5*(2*K0/Lh)
print('%3.2f * 10^-7' % (Kh*1.e7))

# Equations and Solver
problem = de.IVP(domain, variables=['tr','trh'])
problem.meta[:]['h']['dirichlet'] = True

problem.parameters['K0'] = K0
problem.parameters['Kh'] = Kh
problem.parameters['wd'] = wd
problem.add_equation("dt(tr) + (wd-Kh)*trh - K0*dh(trh) = Kh*h*dh(trh)")
problem.add_equation("trh - dh(tr) = 0")

# Build solver
solver = problem.build_solver(de.timesteppers.RK222)
logger.info('Solver built')

# Initial condition:
tr = solver.state['tr']
trh = solver.state['trh']

# Gaussian:
sh = sz; # from 2D problem Initial condition.
#sh = Lh/20.;
ch = 0.;
tr['g'] = np.exp(-(h-ch)**2/2/sh**2)/np.sqrt(2*np.pi*sh**2)
tr.differentiate('h',out=trh)

# From 2D problem:
# tr['g'] = np.interp(h,zF-zF[imax],trF[:,0]/np.sum(trF[:,0]*dzF),left=0.,right=0.)
# #tr['g'][tr['g']<0] = 0.

# Integration parameters
lday = 1.0e5 # A "long-day" unit (86400 ~= 100000)
dt=1*lday
Ttot = 360.*5
solver.stop_sim_time = np.inf
solver.stop_wall_time = np.inf 
solver.stop_iteration = Ttot*lday/dt
sfreq = 2
Itot = solver.stop_iteration

## Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', iter=sfreq, max_writes=20000)
snapshots.add_system(solver.state, layout='g')
snapshots.add_task("integ(tr,'h')", layout='g', name = 'hm0')
snapshots.add_task("integ(tr*h,'h')", layout='g', name = 'hm1')
snapshots.add_task("integ(tr*h*h,'h')", layout='g', name = 'hm2')

In [ ]:
# Setup figure:
h = domain.grid(0,scales=domain.dealias)
f = plt.figure(figsize=(10,10), facecolor='w')
ax = plt.subplot(1,1,1)
p, = ax.plot(tr['g'],h,'-k')
po, = ax.plot(trF[:,0],zF,'-r')
#ax.set_xlim([0.,0.5])
ax.set_ylim([h[0],h[-1]])

# Main loop
try:
    logger.info('Starting loop')
    start_time = time.time()
    while solver.ok:
        solver.step(dt)
        # if (solver.iteration-1) % 100 == 0:
        #     logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))
        if (solver.iteration-1) % 100 == 0:
            p.set_xdata(tr['g'])
            ind = np.argmin(np.abs(tF*lday - solver.iteration*dt))
            po.set_xdata(trF[:,ind])
            display.clear_output()
            display.display(plt.gcf())
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    end_time = time.time()

    logger.info('Iterations: %i' %solver.iteration)
    logger.info('Sim end time: %f' %solver.sim_time)
    logger.info('Run time: %.2f sec' %(end_time-start_time))
    logger.info('Run time: %f cpu-hr' %((end_time-start_time)/60/60*domain.dist.comm_cart.size))